# Common blocks

In [6]:
# run once
%matplotlib

Using matplotlib backend: TkAgg


In [7]:
# import modules
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import json
import visa
import myplots_py3, myutils # custom modules
import os.path
import time
from imp import reload # for python3

# instrument drivers
import SIM
import AgilentN5230A
#import HittiteHMC
#import AgilentMXG

# instrument initialization
pna = AgilentN5230A.AgilentN5230A(16)
sim_glb = SIM.SIM(15, channel=3)
sim_chip = SIM.SIM(15, channel=1)
#spec = HittiteHMC.HittiteHMC(18)
#spec = AgilentMXG.AgilentMXG(30)

In [8]:
rm=visa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR',
 'ASRL4::INSTR',
 'ASRL6::INSTR',
 'ASRL9::INSTR',
 'ASRL10::INSTR',
 'ASRL11::INSTR',
 'GPIB0::2::INSTR',
 'GPIB0::6::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::11::INSTR',
 'GPIB0::15::INSTR',
 'GPIB0::16::INSTR',
 'GPIB0::20::INSTR',
 'GPIB0::30::INSTR')

In [21]:
sim_glb.setVoltage(0)  #-0.45
sim_chip.setVoltage(0)

# S21 vs Freq sweep, continuous

In [25]:
####################################################################################
# set up parameter
startFreq, endFreq = 6.5e9,7.5e9
power = -20
numPoints, IFbandwidth, avg = 2001, 500, 3
#volt = 0 # -0.65 # -2.05  #2.4
####################################################################################

# plot setup
plotobj= myplots_py3.plot1D(ylabel = 'S21 (dB)', xlabel='Frequency (Hz)', title = 'S21 vs Freq'
                        , plotStyle='b-')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')

pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

#spec.pwrOff()
#sim_glb.setVoltage(volt)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

################ sweep start ################

for i in range(1):
    # configure
    pna.avgClear()
        
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    # plot update
    plotobj.update(freq/1e9, S21_mag_log)
    
     
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
print('Measurement Done!!')


Initialized S21 Measurement 'S21Meas'
Measurement Done!!


In [11]:
reload(myplots_py3)

<module 'myplots_py3' from 'Z:\\User\\Jaseung\\programs\\pythonCode\\PNA\\myplots_py3.py'>

# S21 vs Freq vs Power, 2D

In [28]:
%%time
#########################################################################################
# set up parameters
startFreq, endFreq = 7.096e9, 7.100e9 #6.775e9 , 6.945e9 #6.881e9, 6.885e9 # 6.904e9, 6.912e9
numPoints, avg = 801, 1
IFStart, IFEnd =200, 500 # Hz
startPwr, endPwr, stepPwr = -70,-20, 2
volt_glb_initial = 3
volt_chip_initial = 0

#save_path = 'Z:\User\JJ\SFQ2\C6' # no backslash at the end.
save_path = r'Z:\User\Jaseung\projects\CSFQ\measurement\W5\B3\DR1\S21vsFreqvsP'
#save_path = 'C:\Users\Lab User\Desktop\Albany1222'

#surfix='Q3_Vglb=0.328V' # surfix to the file name
surfix='Q6_Vglb=3_Vchip=0V'  # surfix to file name
commentStr=''  # comment to be placed in the log file.
############################################################################################

# IFbandwidth scales up linearly as power increases.
IF_arr = np.linspace(IFStart, IFEnd, np.arange(startPwr, endPwr + stepPwr, stepPwr).size) 

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Power (dBm)', startPwr, endPwr, stepPwr
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd)
             , ('numPoints',numPoints), ('IFbandwidth start (Hz)', IF_arr[0]),('IFbandwidth end (Hz)', IF_arr[-1])
             , ('avg',avg), ('volt_glb_initial (V)',volt_glb_initial),('volt_chip_initial (V)',volt_chip_initial), ('comment', commentStr)])

# file setup
timeStr = time.strftime("%Y%m%d_%H%M%S")
baseStr = 'S21vsFvsP_fr{:.9g}_{:.9g}_Pr{}_{}_{}_{}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, startPwr, endPwr, stepPwr,surfix, timeStr)
dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + '_mag.dat'), 
                                         os.path.join(save_path, baseFileName + '_phase.dat'))
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

#with open(logFileName, 'wb') as f_log:
#    for k, v in para.items():  # write log file.
#        f_log.write('# {} : {} \n'.format(k,v))
   
# plot setup
plotobj= myplots_py3.plot2D(ylabel = 'Frequency(Hz)', xlabel='Power (dBm)', title = 'S21 vs Freq vs Power')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
#pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
#pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(volt_glb_initial)
sim_chip.setVoltage(volt_chip_initial)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])
i=0
################ sweep start ################
for pwr in np.arange(startPwr, endPwr + stepPwr, stepPwr): # traverse in [pmin, pmax].
    
    clear_output()
    print('startPwr={:.3g} dBm, endPwr={:.3g} dBm, current power={:.3g} dBm'.format(startPwr, endPwr, pwr))
    
    # configure
    pna.setIF(IF_arr[i]) 
    pna.setPwr(pwr)
    pna.avgClear()
    i += 1
    
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
        S21_mag_log_2D = S21_mag_log
        S21_mag_log_2D.shape = (S21_mag_log.size, 1)
        S21_phase_2D = S21_phase
        S21_phase_2D.shape = (S21_phase.size, 1)
    else:
        S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
        S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))
     
    # plot update
    extent = [startPwr, pwr, startFreq, endFreq]
    plotobj.update(S21_mag_log_2D, extent)
    
    
    # save data in each iteration
    f_mag = open(dataFileName_mag, 'wb') 
    f_phase = open(dataFileName_phase, 'wb') 
    np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
    
############## sweep end #########################    

# Post measurement  
del S21_mag_log_2D, S21_phase_2D
pna.pwrOff()
print('Measurement complete!!!')


startPwr=-70 dBm, endPwr=-20 dBm, current power=-20 dBm
Measurement complete!!!
Wall time: 1min 6s


# S21 vs Freq vs Flux, 2D

In [37]:
%%time
###########################################################################
# set up parameters
startFreq, endFreq = 7.097e9, 7.105e9 # 6.935e9, 6.9450e9 #6.7949e9 , 6.9676e9 #6.83e9, 6.894e9#6.856e9, 7.015e9 #6.85e9, 7.05e9
power = -60
numPoints, IFbandwidth, avg = 801, 500, 1
startVolt, endVolt, stepVolt = -20, 20,1  #-20, 20, 0.2 #-5, 5, 0.05 #  #-15, 15, 0.4#-20, 20, 0.5 #-1, 1.5, 0.04
sweepSrc = 2   # 1: Vglb, 2: Vchip
Vglb_init = 0.5
Vglb_endV = 0  # end voltage for both global and onchip flux
Vchip_init = 0 #0.625
Vchip_endV = 0 #0.625
#save_path = 'Z:\User\JJ\SFQ2\C6'
#save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017\S21vsFreqvsV'
save_path =r'Z:\User\Jaseung\projects\CSFQ\measurement\W5\B3\DR1\S21vsFreqvsV'
#surfix = 'Q6_Vchip=0V'
surfix = 'Q6_Vglb=0.5V'
commentStr =''
############################################################################

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Volt (V)', startVolt, endVolt, stepVolt
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd), ('Power (dBm)', power)
             , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
             , ('avg',avg), ('comment', commentStr)])

# file setup

baseStr = 'S21vsFvsV_fr{:.9g}_{:.9g}_Pr{:.4g}_V{}_{}_{}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, power, startVolt, endVolt, stepVolt, surfix)
dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + '_mag.dat'), 
                                         os.path.join(save_path, baseFileName + '_phase.dat'))
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

# plot setup
plotobj= myplots_py3.plot2D(ylabel = 'Frequency (Hz)', xlabel='Volt (V)', title = 'S21 vs Freq vs Volt')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(Vglb_init)
sim_chip.setVoltage(Vchip_init)
time.sleep(2)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

################ sweep start ################
for volt in np.arange(startVolt, endVolt + stepVolt, stepVolt): # traverse in [pmin, pmax].
    clear_output()
    print('startV={:.3g} V, endV={:.3g} V, current volt={:.3g} V'.format(startVolt, endVolt, volt))

    # configure
    if sweepSrc==1:
        sim_glb.setVoltage(volt)
    else: #sweepSrc=2
        sim_chip.setVoltage(volt)
        
    time.sleep(1)
    pna.avgClear()
    
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
        S21_mag_log_2D = S21_mag_log
        S21_mag_log_2D.shape = (S21_mag_log.size, 1)
        S21_phase_2D = S21_phase
        S21_phase_2D.shape = (S21_phase.size, 1)
    else:
        S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
        S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))
     
    # plot update
    extent = [startVolt, volt, startFreq, endFreq]
    plotobj.update(S21_mag_log_2D, extent)
        
    # save data in each iteration
#     f_mag = open(dataFileName_mag, 'w') 
#     f_phase = open(dataFileName_phase, 'w') 
#     np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
#     np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
#     f_mag.close()
#     f_phase.close()
    
############## sweep end #########################    
# save at the end
f_mag = open(dataFileName_mag, 'wb') 
f_phase = open(dataFileName_phase, 'wb') 
np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
f_mag.close()
f_phase.close()

# Post measurement   
del S21_mag_log_2D, S21_phase_2D
pna.pwrOff()
sim_glb.setVoltage(Vglb_endV)
sim_chip.setVoltage(Vchip_endV)
print('Measurement done!!!')

startV=-20 V, endV=20 V, current volt=20 V
Measurement done!!!
Wall time: 2min 5s


In [12]:
extent[2]

6760000000.0

## S21 vs Freq vs Flux, 3D

In [6]:
%%time
###########################################################################
# set up parameters
startFreq, endFreq = 6.77e9, 6.81e9 # 6.935e9, 6.9450e9 #6.7949e9 , 6.9676e9 #6.83e9, 6.894e9#6.856e9, 7.015e9 #6.85e9, 7.05e9
power = -60
numPoints, IFbandwidth, avg = 801, 1000, 1
startVolt, endVolt, stepVolt = -20, 20, 0.2 #-5, 5, 0.05 #  #-15, 15, 0.4#-20, 20, 0.5 #-1, 1.5, 0.04
endV = 0  # end voltage for both global and onchip flux
startVolt1, endVolt1, stepVolt1 =-5.3, 0.7, 0.05

slop, Vglb_offset = -0.0659, -0.5  # Vglb = slope * Vchip + Vglb_offset. Vglb_offset is the y-intercept in Vchip(x-axxis) vs Vglb (y) axis.

#save_path = 'Z:\User\JJ\SFQ2\C6'
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017\S21vsFreqvsV\AlongNormVchipAxis'
surfix = 'Q1_VchipAxis'
commentStr= 'slope={:.3g}, Vglb_offset={:.3g}'.format(slope, Vglb_offset)

############################################################################

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Volt (V)', startVolt, endVolt, stepVolt
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
zID, zStart, zEnd, zStep = 'Volt (V)', startVolt1, endVolt1, stepVolt1
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd)
             ,('zID', zID), ('zStart', zStart), ('zEnd', zEnd), ('Power (dBm)', power)
             , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
             , ('avg',avg), ('comment', commentStr)])

# log file setup
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'S21vsFvsV_fr{:.9g}_{:.9g}_Pr{:.4g}_V{}_{}_{}_T{:.4g}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, power, startVolt, endVolt, stepVolt, FAA, surfix)
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

# plot setup
#plotobj= myplots_py3.plot2D(ylabel = 'Frequency(Hz)', xlabel='Volt (V)', title = 'S21 vs Freq vs Volt')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(slope * startVolt + Vglb_offset)
sim_chip.setVoltage(startVolt)

time.sleep(2)

################ sweep start ################
for Vglb_offset in np.arange(startVolt1, endVolt1+stepVolt1, stepVolt1):
    # data initialization
    S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

    #sim_glb.setVoltage(volt_outer)
    
    for volt_inner in np.arange(startVolt, endVolt + stepVolt, stepVolt): # traverse in [pmin, pmax].
        clear_output()
        print('Vglb_offset={:.3g}'.format(Vglb_offset))
        print('startV={:.3g} V, endV={:.3g} V, current volt={:.3g} V'.format(startVolt, endVolt, volt_inner))

        # configure
        sim_chip.setVoltage(volt_inner)
        sim_glb.setVoltage(slope * volt_inner + Vglb_offset)
        time.sleep(0.1)
        pna.avgClear()

        # fetch data from PNA
        freq, S21 = pna.getData(cplx=True) # S21 in linear scale

        # S21 conversion
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

        if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
            S21_mag_log_2D = S21_mag_log
            S21_mag_log_2D.shape = (S21_mag_log.size, 1)
            S21_phase_2D = S21_phase
            S21_phase_2D.shape = (S21_phase.size, 1)
        else:
            S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
            S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))

        # plot update
        #extent = [startVolt, volt, startFreq, endFreq]
        #plotobj.update(S21_mag_log_2D, extent)
     
    sim_chip.setVoltage(endV)
    time.sleep(3)
     
    # save at the end of inner loop
    fileIdStr = '_Vglb_offset={:.3g}'.format(Vglb_offset)
    dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + fileIdStr + '_mag.dat'), 
                                             os.path.join(save_path, baseFileName + fileIdStr + '_phase.dat'))
    f_mag = open(dataFileName_mag, 'w') 
    f_phase = open(dataFileName_phase, 'w') 
    np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
############## sweep end #########################    

# Post measurement   
del S21_mag_log_2D, S21_phase_2D
pna.pwrOff()
sim_glb.setVoltage(endV)
sim_chip.setVoltage(endV)
print('Measurement done!!!')

Vglb_offset=0.7
startV=-20 V, endV=20 V, current volt=20 V
Measurement done!!!
Wall time: 8h 42min 17s


# S21 vs flux1 vs flux2, 2D

In [27]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
import sys
for i in dir():
    try:
        print (i, eval(i).nbytes )
    except:
        print (i, sys.getsizeof(eval(i)) )

clear_output()

In [52]:
reload(SIM)

<module 'SIM' from 'Z:\\User\\Jaseung\\programs\\pythonCode\\PNA\\SIM.py'>

In [43]:
%%time
def s21vsV1vsV2(readoutFreq):
    ##############################################################################
    # set up parameters
    readoutPwr = -60
    #readoutFreq = 6.7736e9 #6.7696e9 #6.7645e9
    numPoints, IFbandwidth, avg = 50, 500, 1
    startVolt1, endVolt1, stepVolt1 =-7, 7, 0.4#-10, 10, 0.2 #-10, 10, 0.05 #-1.2, 1, 0.025 # outer for-loop
    startVolt2, endVolt2, stepVolt2 =-20, 20, 1 #-20, 20, 0.5 #-20, 20, 0.5 # y axis. Inner for-loop.
    endVolt = 0

    surfix = 'Q1_V2=onchip'
    save_path = r'Z:\User\Jaseung\projects\CSFQ\measurement\W5\B3\DR1\S21vsV1vsV2'
    commentStr='with no spec setup'
    ##############################################################################

    xID, xStart, xEnd, xStep = 'Volt1 (V)', startVolt1, endVolt1, stepVolt1
    yID, yStart, yEnd, yStep = 'Volt2 (V)', startVolt2, endVolt2, stepVolt2
    para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
                 , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd)
                 , ('readoutFreq (Hz)',readoutFreq),('power (dBm)',readoutPwr)
                 , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
                 , ('avg',avg), ('comment', commentStr)])

    # file setup
    baseStr = 'S21vsV1vsV2_fr{:.9g}_Pr{:.4g}_V1_{}_{}_{}_V2_{}_{}_{}_{}'
    baseFileName = baseStr.format(readoutFreq/1e9, readoutPwr, startVolt1, endVolt1, stepVolt1, startVolt2, endVolt2, stepVolt2,surfix)
    dataFileName1  = os.path.join(save_path, baseFileName + '_mag.dat')
    dataFileName2  = os.path.join(save_path, baseFileName + '_phase.dat')
    logFileName =  os.path.join(save_path, baseFileName + '.json')
    with open(logFileName, 'w') as f_log:
        json.dump(para, f_log, indent=4, sort_keys=True)

    # plot setup
    plotobj= myplots_py3.plot2D_4sub(xlabel1 = 'Voltage2 (V)', ylabel1='S21 (dB)', 
                                 xlabel2 = 'Voltage2 (V)', ylabel2='Phase(degree)',
                                 xlabel3 = 'Voltage, Vglb (V)', ylabel3='Voltage, Vchip (V)',
                                 xlabel4 = 'Voltage, Vglb (V)', ylabel4='Voltage, Vchip (V)',
                                 title = 'S21 vs Global Flux vs Gradiometric Flux', figsize=(12,8))

    # preconfigure instrument
    pna.setupMeas()
    pna.setSweepType(mode='CW')
    pna.setCWFreq(readoutFreq)
    pna.setPwr(readoutPwr)
    pna.avgCount(avg)
    pna.numPoints(numPoints)
    pna.setIF(IFbandwidth)
    pna.pwrOn()

    sim_glb.setVoltage(startVolt1)
    time.sleep(2)
    sim_chip.setVoltage(startVolt2)
    time.sleep(2)

    # data initialization
    data1_2D, data2_2D = np.array([]), np.array([])

    ################ sweep start ################
    #for specFreq in arange(startFreq, endFreq + stepFreq, stepFreq): # traverse in [pmin, pmax].
    for volt1 in np.around(np.arange(startVolt1, endVolt1 + stepVolt1, stepVolt1), decimals=3) :
        #configure para
        sim_glb.setVoltage(volt1)
        clear_output()
        print('startVolt1={:.3g}, endtvolt1={:.3g}'.format(startVolt1, endVolt1))
        print('curren volt1={:.3g}'.format(volt1))

        # data initialization
        xdata1, ydata1, ydata2 = np.array([]), np.array([]), np.array([])

        for volt2 in np.around(np.arange(startVolt2, endVolt2 + stepVolt2, stepVolt2), decimals=3):
    #         clear_output()
    #         print('startVolt1={:.3g}, endtvolt1={:.3g}'.format(startVolt1, endVolt1))
    #         print('startVolt2={:.3g}, endtvolt2={:.3g}'.format(startVolt2, endVolt2))
    #         print('volt1={:.3g}, volt2={:.3g}'.format(volt1, volt2)) 

            # configure
            pna.avgClear()
            sim_chip.setVoltage(volt2)

            # fetch data from PNA
            freq, S21= pna.getData(cplx=True) # S21 in linear scale

            # S21 conversion
            S21_real, S21_imag = np.real(S21), np.imag(S21)
            S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
            S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

            S21_mag_log_avg = np.average(S21_mag_log) 
            S21_phase_avg = np.average(S21_phase)

            xdata1 = np.append(xdata1, volt2)
            ydata1 = np.append(ydata1, S21_mag_log_avg)
            ydata2 = np.append(ydata2, S21_phase_avg)

            # plot and update
            plotobj.update1D(xdata1, ydata1, xdata1, ydata2)


        if data1_2D.size == 0: # when S21_mag_2D is empty
            data1_2D = ydata1 # make 2D array
            data1_2D.shape = (data1_2D.size, 1)
            data2_2D = ydata2
            data2_2D.shape = (data2_2D.size, 1)

        else:
            data1_2D = np.column_stack((data1_2D, ydata1))
            data2_2D = np.column_stack((data2_2D, ydata2))

        # plot and update
        extent = [startVolt1, volt1, startVolt2, endVolt2]
        plotobj.update2D(data1_2D, data2_2D, extent, cmap='Blues_r')

        # save data in each iteration
        f_mag = open(dataFileName1, 'wb') 
        f_phase = open(dataFileName2, 'wb') 
        np.savetxt(f_mag, data1_2D, fmt='%.9g', delimiter='\t') 
        np.savetxt(f_phase, data2_2D, fmt='%.9g', delimiter='\t')
        f_mag.close()
        f_phase.close()

    ############## sweep end #########################    

    # Post measurement    
    pna.pwrOff()
    pna.setSweepType(mode='linear')
    del data1_2D, data2_2D
    sim_glb.setVoltage(endVolt)
    sim_chip.setVoltage(endVolt)
    f_log.close()
    print('Measurement done!!!')

Wall time: 0 ns


In [44]:
%%time
# for cavFreq in np.linspace(6.9975e9, 7.0098e9,6):
#     print('readout freq={:.6g}'.format(cavFreq))
#     s21vsV1vsV2(cavFreq)
s21vsV1vsV2(6.8427e9)
# s21vsV1vsV2(7.0013e9)
# s21vsV1vsV2(7.00242e9)



startVolt1=-7, endtvolt1=7
curren volt1=7
Measurement done!!!
Wall time: 8min 7s


In [15]:
np.linspace(6.7669e9, 6.7927e9,6)

array([  6.76690000e+09,   6.77206000e+09,   6.77722000e+09,
         6.78238000e+09,   6.78754000e+09,   6.79270000e+09])

In [65]:
startVolt1, endVolt1, stepVolt1 =-5, 5, 0.2#-10, 10, 0.2 #-10, 10, 0.05 #-1.2, 1, 0.025 # outer for-loop
startVolt2, endVolt2, stepVolt2 =-6, 6, 1 #-20, 20, 0.5 #-20, 20, 0.5 # y axis. Inner for-loop.
rm = visa.ResourceManager()
#instr = rm.get_instrument('GPIB0::15DDR) + '::INSTR')
#vrange=np.arange(startVolt1, endVolt1 + stepVolt1, stepVolt1)
#print(vrange)   

for volt1 in np.around(np.arange(startVolt1, endVolt1 + stepVolt1, stepVolt1), decimals=3) :
    #configure para
    sim_chip.setVoltage(volt1)
    #instr.write('SNDT {}, "VOLT {}"'.format(self.channel, value))
    #print('volt1={:.3g}'.format(volt1)) 
    time.sleep(1)

KeyboardInterrupt: 

In [63]:
np.around([3.12356], decimals=4)
#print(aa[1], round(aa[1],3))

array([ 3.1236])

# S21 vs Freq vs Flux (linear comb. of flux1 and flux2), 2D

In [10]:
%%time
def s21vsfreqvsfluxcombined(slope, Vglb_offset):
    ###########################################################################
    # set up parameters
    startFreq, endFreq =  6.75e9, 6.78e9 #6.83e9, 6.894e9#6.856e9, 7.015e9 #6.85e9, 7.05e9
    power = -50
    numPoints, IFbandwidth, avg = 801, 1000, 1
    startVolt, endVolt, stepVolt =-15, 15, 0.25#-20, 20, 0.5 #-1, 1.5, 0.04
    endV = 0

    #slope, Vglb_offset =-1.85/22.5, 2.4 # 0.2107, 0# 2.484#-0.3378, -0.9641# -0.0659, 0.7   # offset means Vglb when Vchip=0

    #save_path = 'Z:\User\JJ\sput\M5_1_spec'
    save_path = r'Z:\User\Jaseung\projects\CSFQ\measurement\W4\B5\BlueFors\S21vsFreqvsV'
    surfix = 'Q1_xID=Vchip_slope{:.3g}_Vglb_offset{:.3g}'.format(slope, Vglb_offset)
    comment= 'slope={:.3g}, Vglb_offset={:.3g}'.format(slope, Vglb_offset)

    plottitle = 'S21 vs Freq vs Vchip, slope={:.3g}, Vglb_offset={:.3g}'.format(slope,Vglb_offset)

    ############################################################################

    # dictionary for parameters to be saved in json file. Solely for log.
    xID, xStart, xEnd, xStep = 'Volt (slope, Vglb_offset)', startVolt, endVolt, stepVolt
    yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
    para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
                 , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd), ('Power (dBm)', power)
                 , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
                 , ('avg',avg), ('comment', comment)])

    # file setup
    baseStr = 'S21vsFvsV_fr{:.9g}_{:.9g}_Pr{:.4g}_V{}_{}_{}_{}'
    baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, power, startVolt, endVolt, stepVolt, surfix)
    dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + '_mag.dat'), 
                                             os.path.join(save_path, baseFileName + '_phase.dat'))
    logFileName =  os.path.join(save_path, baseFileName + '.json')
    with open(logFileName, 'w') as f:
        json.dump(para,f,indent=4, sort_keys=True)

    # plot setup
    plotobj= myplots_py3.plot2D(ylabel = 'Frequency(Hz)', xlabel='Volt (V)', title = plottitle)

    # preconfigure instrument
    pna.setupMeas()
    pna.setSweepType('linear')
    pna.setFreqStartEnd(startFreq, endFreq)
    pna.setPwr(power)
    pna.avgCount(avg)
    pna.numPoints(numPoints)
    pna.setIF(IFbandwidth)
    pna.pwrOn()

    sim_glb.setVoltage(slope*startVolt+Vglb_offset)
    sim_chip.setVoltage(startVolt)
    time.sleep(2)

    # data initialization
    S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

    ################ sweep start ################
    for volt_chip in np.around(np.arange(startVolt, endVolt + stepVolt, stepVolt), decimals=3): # traverse in [pmin, pmax].
        # configure
        volt_glb =  slope * volt_chip + Vglb_offset
        #volt_glb = -1/slope* (volt_chip-20) - 0.55
        clear_output()
        print('volt_glb={:.3g}, volt_chip={:.3g}'.format(volt_glb, volt_chip))
        sim_chip.setVoltage(volt_chip)
        sim_glb.setVoltage(volt_glb)

        time.sleep(0.1)
        pna.avgClear()

        # fetch data from PNA
        freq, S21 = pna.getData(cplx=True) # S21 in linear scale

        # S21 conversion
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

        if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
            S21_mag_log_2D = S21_mag_log
            S21_mag_log_2D.shape = (S21_mag_log.size, 1)
            S21_phase_2D = S21_phase
            S21_phase_2D.shape = (S21_phase.size, 1)
        else:
            S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
            S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))

        # plot update
        extent = [startVolt, volt_chip, startFreq, endFreq]
        plotobj.update(S21_mag_log_2D, extent)

        # save data in each iteration
        f_mag = open(dataFileName_mag, 'wb') 
        f_phase = open(dataFileName_phase, 'wb') 
        np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
        np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
        f_mag.close()
        f_phase.close()

    ############## sweep end #########################    

    # Post measurement   
    del S21_mag_log_2D, S21_phase_2D
    pna.pwrOff()
    sim_glb.setVoltage(endV)
    sim_chip.setVoltage(endV)
    print('Measurement done!!!')

Wall time: 0 ns


In [9]:
#s21vsfreqvsfluxcombined(-1.85/22.5,  -0.372222)  #A line

#s21vsfreqvsfluxcombined(1.55/21,  -0.535714)  # B line
#s21vsfreqvsfluxcombined(-0.1/22, 0.404545)  # D line

volt_glb=0.336, volt_chip=15
Measurement done!!!


In [1]:
import SIM

In [2]:
sim_glb = SIM.SIM(18, channel=3)
sim_chip = SIM.SIM(19, channel=1)

In [3]:
sim_glb.setVoltage(0.7)
sim_chip.setVoltage(0.625)